<a href="https://colab.research.google.com/github/GawainGan/Machine-Learning-Engineering-for-Production-Specialization/blob/main/2-Machine%20Learning%20Data%20Lifecycle%20in%20Production/W2/C2_W2_Lab_2_Feature_Engineering_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Feature Engineering Pipeline


In this lab, you will continue exploring [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started). This time, it will be in the context of a machine learning (ML) pipeline. In production-grade projects, you want to streamline tasks so you can more easily improve your model or find issues that may arise. [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) provides components that work together to execute the most common steps in a machine learning project. If you want to dig deeper into the motivations behind TFX and the need for machine learning pipelines, you can read about it in [this paper](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b500d77bc4f518a1165c0ab43c8fac5d2948bc14.pdf) and in this [blog post](https://blog.tensorflow.org/2020/09/brief-history-of-tensorflow-extended-tfx.html).

You will build end-to-end pipelines in future courses but for this one, you will only build up to the feature engineering part. Specifically, you will:

* ingest data from a base directory with `ExampleGen`
* compute the statistics of the training data with `StatisticsGen`
* infer a schema with `SchemaGen`
* detect anomalies in the evaluation data with `ExampleValidator`
* preprocess the data into features suitable for model training with `Transform`

If several steps mentioned above sound familiar, it's because the TFX components that deal with data validation and analysis (i.e. `StatisticsGen`, `SchemaGen`, `ExampleValidator`) uses [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) under the hood. You're already familiar with this library from the exercises in Week 1 and for this week, you'll see how it fits within an ML pipeline.

The components you will use are the orange boxes highlighted in the figure below:

<img src='img/feature_eng_pipeline.png'>



## Setup

### Import packages

Let's begin by importing the required packages and modules. In case you want to replicate this in your local workstation, we used *Tensorflow v2.6* and *TFX v1.3.0*.

In [ ]:
import tensorflow as tf

from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

### Define paths

You will define a few global variables to indicate paths in the local workspace.

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/census_data'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'adult.data')

### Preview the  dataset

You will again be using the [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult) from the Week 1 ungraded lab so you can compare outputs when just using stand-alone TFDV and when using it under TFX. Just to remind, the data can be used to predict if an individual earns more than or less than 50k US Dollars annually. Here is the description of the features again:


* **age**: continuous.
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* **fnlwgt**: continuous.
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* **education-num**: continuous.
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* **sex**: Female, Male.
* **capital-gain**: continuous.
* **capital-loss**: continuous.
* **hours-per-week**: continuous.
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
# preview the first few rows of the CSV file
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0,

### Create the Interactive Context

When pushing to production, you want to automate the pipeline execution using orchestrators such as [Apache Beam](https://beam.apache.org/) and [Kubeflow](https://www.kubeflow.org/). You will not be doing that just yet and will instead execute the pipeline from this notebook. When experimenting in a notebook environment, you will be *manually* executing the pipeline components (i.e. you are the orchestrator). For that, TFX provides the [Interactive Context](https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/experimental/interactive/interactive_context.py) so you can step through each component and inspect its outputs.


创建交互式上下文


当将模型推送到生产环境时，您会希望使用诸如 Apache Beam 和 Kubeflow 这样的编排工具来自动化管道执行。但在当前阶段，您还不会这样做，而是将从这个笔记本中执行管道。在笔记本环境中进行实验时，您将手动执行管道组件（即您是编排者）。为此，TFX 提供了 交互式上下文，以便您可以逐步执行每个组件并检查其输出。

You will initialize the `InteractiveContext` below. This will create a database in the `_pipeline_root` directory which the different components will use to save or get the state of the component executions. You will learn more about this in Week 3 when we discuss ML Metadata. For now, you can think of it as the data store that makes it possible for the different pipeline components to work together.

*Note: You can configure the database to connect to but for this exercise, we will just use the default which is a newly created local sqlite file.* ***You will see the warning after running the cell below and you can safely ignore it.***

您将在下面初始化 InteractiveContext。这将在 _pipeline_root 目录中创建一个数据库，不同的组件将使用该数据库来保存或获取组件执行的状态。在第三周的课程中，当我们讨论机器学习元数据（ML Metadata）时，您将了解到更多相关信息。目前，您可以将其视为使不同管道组件能够协同工作的数据存储。

注意：您可以配置要连接的数据库，但在本练习中，我们将使用默认设置，即新创建的本地 sqlite 文件。 在运行下面的单元格后，您将看到一个警告，您可以放心地忽略它。

In [ ]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

# InteractiveContext 自带sqlite数据库
# pipeline_root= 的default时临时dict
# 设置pipeline_root=_pipeline_root 代表在_pipeline_root的路径下创建数据库

TensorFlow Extended (TFX) 的 InteractiveContext 来初始化一个交互式的机器学习管道环境。

这个环境特别适用于在笔记本或交互式环境中进行实验和开发。

- InteractiveContext：
    - InteractiveContext 是 TFX 提供的一个工具，允许您在交互式环境（如Jupyter笔记本）中逐步运行和测试各个管道组件。
    - 它提供了一个方便的界面来执行和检查管道的各个阶段，而无需设置完整的管道编排和部署。

- 本地 SQLite 文件：
    - 当您初始化 InteractiveContext 时，它会使用一个 SQLite 数据库来存储管道的元数据。
    - 元数据包括管道运行的信息，如执行状态、输出结果等。

- _pipeline_root：
    - _pipeline_root 是一个变量，指定了存储管道产生的数据和元数据的根目录。
    - 如果 _pipeline_root 为空，SQLite 数据库将被创建在一个临时目录中。
    - 在这个例子中，通过设置 pipeline_root=_pipeline_root，您指定了数据库应该在 _pipeline_root 指定的路径下创建。

## Run TFX components interactively

With that, you can now run the pipeline interactively. You will see how to do that as you go through the different components below.

### ExampleGen

You will start the pipeline with the [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component. This  will:

*   split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
*   convert each data row into `tf.train.Example` format. This [protocol buffer](https://developers.google.com/protocol-buffers) is designed for Tensorflow operations and is used by the TFX components.
*   compress and save the data collection under the `_pipeline_root` directory for other components to access. These examples are stored in `TFRecord` format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

Its constructor takes the path to your data source/directory. In our case, this is the `_data_root` path. The component supports several data sources such as CSV, tf.Record, and BigQuery. Since our data is a CSV file, we will use [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) to ingest the data.

Run the cell below to instantiate `CsvExampleGen`.

您将从 ExampleGen 组件开始构建管道。这个组件将会：

- 将数据分割为训练集和评估集（默认比例为：2/3 训练，1/3 评估）。
- 将每行数据转换为 tf.train.Example 格式。这种协议缓冲区是为Tensorflow操作设计的，并被TFX组件使用。
- 压缩并保存数据集到 _pipeline_root 目录下，以供其他组件访问。
这些示例存储在 TFRecord 格式中。如果您有大量数据，这种格式可以优化Tensorflow内的读写操作。

ExampleGen 组件的构造函数需要您提供数据源/目录的路径。在我们的案例中，这就是 _data_root 路径。该组件支持多种数据源，如 CSV、tf.Record 和 BigQuery。由于我们的数据是 CSV 文件，我们将使用 CsvExampleGen 来导入数据。

运行下面的单元格以实例化 CsvExampleGen。

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
example_gen

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7f7e4cd317f0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f7e4cd31a30>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

You can execute the component by calling the `run()` method of the `InteractiveContext`.

In [ ]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 9
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 13
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3974460,xor_checksum:1635233824,sum_checksum:1635233824"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1705445371087
        last_update_time_since_epoch: 1705445382338
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

You will notice that an output cell showing the execution results is automatically shown. This metadata is recorded into the database created earlier. This allows you to keep track of your project runs. For example, if you run it again, you will notice the `.execution_id` incrementing.

The output of the components are called *artifacts* and you can see an example by navigating through  `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` above. It shows information such as where the converted data is stored (`.uri`) and the splits generated (`.split_names`).

You can also examine the output artifacts programmatically with the code below.


您会注意到，执行结果会自动显示在一个输出单元格中。这些元数据被记录到之前创建的数据库中。这使您能够跟踪项目的运行情况。例如，如果您再次运行它，您会注意到 .execution_id 会递增。

组件的输出被称为工件（artifacts），您可以通过导航到 `.component.outputs > ['examples'] > Channel > ._artifacts > [0] `来查看一个示例。它显示了诸如转换后的数据存储位置（.uri）和生成的数据分割（.split_names）等信息。

您也可以使用下面的代码以编程方式检查输出工件。

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/6


If you're wondering , the `number` in `./pipeline/CsvExampleGen/examples/{number}` is the execution id associated with that dataset. If you restart the kernel of this workspace and re-run up to this cell, you will notice a new folder with a different id name created. This shows that TFX is keeping versions of your data so you can roll back if you want to investigate a particular execution.

如果您好奇，./pipeline/CsvExampleGen/examples/{number} 中的 number 是与该数据集相关联的执行 ID。如果您重启这个工作区的内核并重新运行到这个单元格，您会注意到创建了一个带有不同 ID 名称的新文件夹。这表明 TFX 正在保留您数据的不同版本，以便您可以回滚，如果您想调查某个特定的执行情况。

As mentioned, the ingested data is stored in the directory shown in the `uri` field. It is also compressed using `gzip` and you can verify by running the cell below.

正如所提到的，摄取的数据存储在 uri 字段中显示的目录中。数据还使用 gzip 进行了压缩，您可以通过运行下面的单元格来验证这一点。

In [ ]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In a notebook environment, it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. Let's do that in the following cells:

在笔记本环境中，如果您仍在进行实验，检查一些数据示例可能会很有用。由于数据集是以 TFRecord 格式保存的，您需要使用适用于该数据类型的方法。您需要从 TFRecord 文件中解包单个示例，并将其格式化以便打印。让我们在接下来的单元格中执行这些操作：

In [ ]:
# Get the list of files in this directory (all compressed TFRecord files)
# 在这段代码中，您将获取存储在特定目录中的所有压缩 TFRecord 文件的列表，并创建一个 TFRecordDataset 来读取这些文件。
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

解释上面的代码:
1. 获取 TFRecord 文件列表：
    - 使用 os.path.join 和 os.listdir 函数来构建包含目录中所有 TFRecord 文件路径的列表。
    - train_uri 是存储 TFRecord 文件的目录路径。
    - tfrecord_filenames 将是一个包含所有 TFRecord 文件完整路径的列表。

2. 创建 TFRecordDataset：
    - tf.data.TFRecordDataset 用于读取 TFRecord 文件。
    - 通过传递 tfrecord_filenames 列表，您可以创建一个能够读取所有指定 TFRecord 文件的数据集。
    - compression_type="GZIP" 参数指定了文件是以 GZIP 格式压缩的，这对于读取压缩的 TFRecord 文件是必要的。

In [ ]:
# 验证上面的读取的dataset从哪个路径来的
# 打印每个 TFRecord 文件的路径
for filename in tfrecord_filenames:
    print(filename)

./pipeline/CsvExampleGen/examples/6/Split-train/data_tfrecord-00000-of-00001.gz


In [ ]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''

    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):

        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()

        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()

        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)

        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))

        # append to the records list
        records.append(example_dict)

    return records

In [ ]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'age': {'int64List': {'value': ['39']}},
                           'capital-gain': {'int64List': {'value': ['2174']}},
                           'capital-loss': {'int64List': {'value': ['0']}},
                           'education': {'bytesList': {'value': ['IEJhY2hlbG9ycw==']}},
                           'education-num': {'int64List': {'value': ['13']}},
                           'fnlwgt': {'int64List': {'value': ['77516']}},
                           'hours-per-week': {'int64List': {'value': ['40']}},
                           'label': {'bytesList': {'value': ['IDw9NTBL']}},
                           'marital-status': {'bytesList': {'value': ['IE5ldmVyLW1hcnJpZWQ=']}},
                           'native-country': {'bytesList': {'value': ['IFVuaXRlZC1TdGF0ZXM=']}},
                           'occupation': {'bytesList': {'value': ['IEFkbS1jbGVyaWNhbA==']}},
                           'race': {'bytesList': {'value': ['IFdoaXRl']}},
                    

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen
The [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component computes statistics over your dataset for data analysis, as well as for use in downstream components (i.e. next steps in the pipeline). As mentioned earlier, this component uses TFDV under the hood so its output will be familiar to you.

`StatisticsGen` takes as input the dataset we just ingested using `CsvExampleGen`.

StatisticsGen 组件

StatisticsGen 组件用于计算您的数据集的统计信息，用于数据分析以及在管道的下游组件中使用。如前所述，这个组件在底层使用了 TensorFlow Data Validation (TFDV)，因此它的输出将对您来说是熟悉的。

StatisticsGen 将使用 CsvExampleGen 导入的数据集作为输入。这意味着它会接收 CsvExampleGen 组件处理后的数据，并在这些数据上计算各种统计信息。这些统计信息包括但不限于每个特征的分布、均值、标准差、最大值和最小值等。这些信息对于理解数据集的特性、检测数据中的异常值和缺失值以及优化后续的数据处理和模型训练步骤至关重要。

通过计算和分析这些统计信息，StatisticsGen 为机器学习管道中的数据准备和特征工程提供了重要的洞察。

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 10
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 14
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

解释上面的代码:

在这段代码中，您将实例化 StatisticsGen 组件，并使用 ExampleGen 组件摄取的数据集作为输入。然后执行该组件以计算数据集的统计信息。

1. 实例化 StatisticsGen:
     - 创建 StatisticsGen 组件的实例，命名为 statistics_gen。
     - 将 example_gen.outputs['examples'] 作为输入传递给 StatisticsGen。这里的 example_gen 是之前使用 CsvExampleGen 创建的实例，其输出包含了转换后的数据集。

2. 执行组件:
     - 使用 context.run(statistics_gen) 执行 StatisticsGen 组件。这里的 context 是之前创建的 InteractiveContext 实例，用于在交互式环境中运行 TFX 组件。
     - 执行这个步骤将会计算数据集的统计信息，并将结果保存在内部数据库中，供后续的管道组件使用。
     - 执行 StatisticsGen 组件是数据分析和特征工程的重要步骤，因为它提供了数据集的详细统计视图，帮助您更好地理解数据并为后续的模型训练做准备。

You can display the statistics with the `show()` method.

*Note: You can safely ignore the warning shown when running the cell below.*

In [ ]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

The [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component also uses TFDV to generate a schema based on your data statistics. As you've learned previously, a schema defines the expected bounds, types, and properties of the features in your dataset.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

SchemaGen 组件

SchemaGen 组件同样使用 TensorFlow Data Validation (TFDV) 基于您的数据统计信息生成架构。正如您之前所学的，架构定义了数据集中特征的预期边界、类型和属性。

SchemaGen 将使用我们通过 StatisticsGen 生成的统计信息作为输入，默认情况下，它会查看训练数据分割。这意味着 SchemaGen 将分析训练数据的统计信息，并据此生成一个架构，这个架构将捕捉每个特征的特性，如数据类型（例如，浮点数、整数、字符串）、值的范围、是否存在缺失值等。

生成的架构对于确保数据的一致性和质量非常重要，它可以用于验证新数据是否符合预期的格式和分布，也可以在后续的数据处理和模型训练中作为一个重要的参考。通过自动化地从数据统计信息中生成架构，SchemaGen 为机器学习管道提供了一个强大的工具，以确保数据的一致性和质量。

In [ ]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 11
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 15
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/11"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

You can then visualize the generated schema as a table.

In [ ]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'fnlwgt',INT,required,,-
'hours-per-week',INT,required,,-
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


Let's now move to the next step in the pipeline and see if there are any anomalies in the data.

### ExampleValidator

The [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component detects anomalies in your data based on the generated schema from the previous step. Like the previous two components, it also uses TFDV under the hood.

`ExampleValidator` will take as input the statistics from `StatisticsGen` and the schema from `SchemaGen`. By default, it compares the statistics from the evaluation split to the schema from the training split.

ExampleValidator 组件

ExampleValidator 组件基于上一步生成的架构来检测数据中的异常情况。与前两个组件一样，它也在底层使用 TensorFlow Data Validation (TFDV)。

ExampleValidator 将从 StatisticsGen 获取统计信息和从 SchemaGen 获取的架构作为输入。默认情况下，它会将评估数据分割的统计信息与训练数据分割的架构进行比较。

这个过程的目的是识别数据中可能存在的任何异常或不一致性，例如：

    - 数据中的特征值超出了在架构中定义的预期范围。
    - 数据类型与架构中定义的类型不匹配。
    - 缺失的特征或出现了架构中未定义的新特征。
    
通过检测这些异常，ExampleValidator 能够帮助确保评估数据集的质量与训练数据集保持一致，从而减少模型训练和评估过程中可能出现的问题。这对于构建可靠和健壮的机器学习模型至关重要。

In [ ]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 13
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 16
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/12"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1705447156018
        last_update_time_since_epoch: 1705447156445
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

As with the previous component, you can also visualize the anomalies as a table.

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

With no anomalies detected, you can proceed to the next step in the pipeline.

### Transform
The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering for both training and serving datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library introduced in the first ungraded lab of this week.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, you will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

Transform 组件

Transform 组件用于对训练和服务数据集执行特征工程。它使用了在本周第一个未评分实验室中介绍的 TensorFlow Transform 库。

Transform 将从 ExampleGen 获取数据，从 SchemaGen 获取架构，以及包含预处理函数的模块作为输入。

在这一部分，您将处理一个用户定义的 Transform 代码示例。管道需要将此代码作为模块加载，因此您需要使用魔法命令 `%% writefile` 将文件保存到磁盘。首先，让我们定义一些常量，这些常量根据我们稍后将执行的转换对数据的属性进行分组。这个文件也将被保存在本地。

通过 Transform 组件，您可以实现复杂的数据转换和特征工程任务，这对于提高模型的性能和适应性至关重要。例如，您可以进行归一化、分箱、创建特征交叉等操作。定义好的转换逻辑将应用于训练数据，并且相同的转换也会在模型部署时应用于新的输入数据，确保了训练和服务之间的一致性。

在实际的机器学习项目中，合理地设计和实现特征工程是成功构建高效、准确模型的关键步骤。通过 Transform 组件，TFX 提供了一个强大的工具，使这一过程更加高效和标准化。

In [ ]:
# Set the constants module filename
_census_constants_module_file = 'census_constants.py'

In [ ]:
%%writefile {_census_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'education', 'marital-status', 'occupation', 'race', 'relationship', 'workclass', 'sex', 'native-country'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['age']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'age': 4}

# Feature that the model will predict
LABEL_KEY = 'label'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting census_constants.py


Next, you will work on the module that contains `preprocessing_fn()`. As you've seen in the previous lab, this function defines how you will transform the raw data into features that your model can train on (i.e. the next step in the pipeline). You will use the [tft module functions](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) to make these transformations.

*Note: After completing the entire notebook, we encourage you to go back to this section and try different tft functions aside from the ones already provided below. You can also modify the grouping of the feature keys in the constants file if you want. For example, you may want to scale some features to `[0, 1]` while others are scaled to the z-score. This will be good practice for this week's assignment.*

接下来，您将处理包含 preprocessing_fn() 的模块。正如您在之前的实验室中所见，这个函数定义了您将如何将原始数据转换为模型可以训练的特征（即管道中的下一步）。您将使用 tft 模块函数 来进行这些转换。

*注意：在完成整个笔记本后，我们鼓励您回到这一部分，并尝试除了下面已提供的 tft 函数之外的其他函数。您也可以修改常量文件中特征键的分组。例如，您可能希望将某些特征缩放到 [0, 1]，而其他特征缩放到 z-分数。这将是本周作业的良好练习。*

preprocessing_fn() 函数是特征工程过程中的核心，它允许您应用各种转换来准备数据，以便更好地适应您的机器学习模型。例如，您可以应用归一化、标准化、分箱、独热编码等转换。通过实验不同的转换方法，您可以探索哪些方法最适合您的数据和模型，从而提高模型的性能和准确性。

在实际应用中，选择合适的数据预处理和特征工程策略对于构建高效的机器学习模型至关重要。TFX 提供的 tft 模块为此提供了丰富的工具和函数，使这一过程更加灵活和高效。

In [ ]:
# Set the transform module filename
_census_transform_module_file = 'census_transform.py'

In [ ]:
%%writefile {_census_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = census_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])

    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting census_transform.py


You can now pass the training data, schema, and transform module to the `Transform` component. You can ignore the warning messages generated by Apache Beam regarding type hints.

In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 15
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 17
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/14"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1705447289532
        last_update_time_since_epoch: 1705447327303
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 18
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/14"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1705447289557
        last_update_time_since_epoch: 1705447327304
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 19
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/14"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1705447289557
        last_update_time_since_epoch: 1705447327305
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 20
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/14"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
       

Let's examine the output artifacts of `Transform` (i.e. `.component.outputs` from the output cell above). This component produces several outputs:

* `transform_graph` is the graph that can perform the preprocessing operations. This graph will be included during training and serving to ensure consistent transformations of incoming data.
* `transformed_examples` points to the preprocessed training and evaluation data.
* `updated_analyzer_cache` are stored calculations from previous runs.

让我们检查 `Transform` 组件的输出工件（即上面输出单元格中的 `.component.outputs`）。这个组件产生了几个输出：

* `transform_graph` 是可以执行预处理操作的图（graph）。这个图将在训练和服务期间被包含进来，以确保对传入数据的一致性转换。
* `transformed_examples` 指向预处理后的训练和评估数据。
* `updated_analyzer_cache` 存储了来自之前运行的计算结果。

这些输出对于理解 `Transform` 组件如何处理数据以及它如何为模型训练和服务做准备至关重要：

1. **transform_graph**：
   - 这是一个 TensorFlow 图，包含了所有定义的数据预处理步骤。
   - 它确保了在模型训练和服务时应用相同的数据转换，从而保持数据的一致性和模型的可靠性。

2. **transformed_examples**：
   - 这是经过预处理的数据，现在以适合模型训练的格式存储。
   - 包括了分割为训练和评估的数据集，这些数据已经应用了所有定义的转换。

3. **updated_analyzer_cache**：
   - 包含了从先前运行中保存的分析结果。
   - 这有助于优化后续运行的效率，因为某些计算可能不需要在每次运行时都重复执行。

通过检查这些输出，您可以更好地理解 `Transform` 组件如何增强数据管道的效率和一致性，并为模型训练和部署做好准备

这些工件为后续的模型训练和评估提供了重要的基础，确保了数据的质量和一致性，从而有助于构建更准确和有效的机器学习模型。

Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [ ]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

* The `metadata` subdirectory contains the schema of the original data.
* The `transformed_metadata` subdirectory contains the schema of the preprocessed data.
* The `transform_fn` subdirectory contains the actual preprocessing graph.

You can also take a look at the first three transformed examples using the helper function defined earlier.

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'age_xf': {'int64List': {'value': ['2']}},
                           'capital-gain_xf': {'floatList': {'value': [0.021740217]}},
                           'capital-loss_xf': {'floatList': {'value': [0.0]}},
                           'education-num_xf': {'floatList': {'value': [0.8]}},
                           'education_xf': {'int64List': {'value': ['2']}},
                           'fnlwgt_xf': {'floatList': {'value': [0.044301897]}},
                           'hours-per-week_xf': {'floatList': {'value': [0.39795917]}},
                           'label_xf': {'int64List': {'value': ['0']}},
                           'marital-status_xf': {'int64List': {'value': ['1']}},
                           'native-country_xf': {'int64List': {'value': ['0']}},
                           'occupation_xf': {'int64List': {'value': ['3']}},
                           'race_xf': {'int64List': {'value': ['0']}},
                           'relationship_xf': {'int64List

**Congratulations!** You have now executed all the components in our pipeline. You will get hands-on practice as well with training and model evaluation in future courses but for now, we encourage you to try exploring the different components we just discussed. As mentioned earlier, a useful exercise for the upcoming assignment is to be familiar with using different `tft` functions in your transform module. Try exploring the [documentation](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) and see what other functions you can use in the transform module. You can also do the optional challenge below for more practice.

**Optional Challenge:** Using this notebook as reference, load the [Seoul Bike Sharing Demand Dataset](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand) and run it through the five stages of the pipeline discussed here. You will first go through the data ingestion and validation components then finally, you will study the dataset's features and transform it to a format that a model can consume. Once you're done, you can visit this [Discourse topic](https://community.deeplearning.ai/t/bike-sharing-dataset-in-c2-w2-lab-2-feature-engineering-pipeline/38979) where one of your mentors, Fabio, has shared his solution. Feel free to discuss and share your solution as well!

**恭喜您！** 您现在已经执行了我们管道中的所有组件。在未来的课程中，您还将亲自实践模型训练和评估，但目前，我们鼓励您尝试探索我们刚刚讨论的不同组件。如前所述，为即将到来的作业做准备的一个有用练习是熟悉在转换模块中使用不同的 `tft` 函数。尝试浏览 [文档](https://www.tensorflow.org/tfx/transform/api_docs/python/tft)，看看您可以在转换模块中使用哪些其他函数。您也可以尝试下面的可选挑战，以获得更多练习。

**可选挑战：** 使用这个笔记本作为参考，加载[首尔自行车共享需求数据集](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand)，并将其运行通过这里讨论的五个管道阶段。您将首先进行数据摄取和验证组件，然后最终研究数据集的特征，并将其转换为模型可以消费的格式。完成后，您可以访问这个 [Discourse 主题](https://community.deeplearning.ai/t/bike-sharing-dataset-in-c2-w2-lab-2-feature-engineering-pipeline/38979)，您的一位导师 Fabio 在那里分享了他的解决方案。也欢迎您讨论和分享您的解决方案！